# Analyse des resultats des test de différentes structures de MLP sur le dataset Cifar-10

### Informations 

Dans ce notebook, nous allons analyser les resultats obtenus sur la première vague de tests de différentes structures et hyperparametres de MLP. Pour chaque un des tests menés, entre 10 et 20 modèles on été testés. 2tant donné la grande quantité d'architectures testées, nous allons mettre seulement un ou deux examples de nos méthodes d'analyse des résutat, et Présenter les résultat définitifs, pour ne pas surcharger le notebook.

## Preparation des outils

In [14]:
import pandas as pd
from IPython.display import display
import numpy as np
from PIL import Image

# --------------------------------------------------------------------------------------------------------------

# MLP Sans régularisation

### Tests réalisés :
    Pour ce début de tests, nous avons essayé d'entrainer différents MLP en jouant sur le nombre de couches ainsi que leur profondeur. Pour cela, on a lancé des tests comme suit (plusieurs fois sur la même structure) :
    - Des modèles de 5 à 10 couches, des profondeurs entre 64 et 512 neurones.
    - Des modèles de 10 à 15 couches, des profondeurs entre 64 et 512 neurones.
    - Des modèles de 15 à 20 couches, des profondeurs entre 64 et 512 neurones.
    - Des modèles de 20 à 30 couches, des profondeurs entre 64 et 512 neurones.
    
    Nous avons ensuite enregistré les résultats obtenus dans des fichiers CSV, en plus des logs générés par tensorflow, pour étudier les résultats obtenus.
    
    Nous allons donc charger ces statistiques et voir ce que ce genre de modèles peut donner sur le Dataset 

## Chargement des résultats obtenus pour chaque configuration de modèle:

In [3]:
results_dataframes_list = []
results_dataframes_desc = []
root_csv_folder = ".\\trained_models\\mlp\\"

# MLP Without regularisation results 5 to 10 Layers
for i in [64, 128, 256, 512]:
    df = pd.read_csv("{}tested_mlp_5_10_{}_history.csv".format(root_csv_folder,i), sep=";")
    df = df[df.epochs == 400]
    results_dataframes_list.append(df)
    results_dataframes_desc.append("mlp_5_10_{}".format(i))

# MLP Without regularisation results 10 to 15 Layers
for i in [64, 128, 256, 512]:
    df = pd.read_csv("{}tested_mlp_10_15_{}_history.csv".format(root_csv_folder,i), sep=";")
    df = df[df.epochs == 400]
    results_dataframes_list.append(df)
    results_dataframes_desc.append("mlp_10_15_{}".format(i))

# MLP Without regularisation results 15 to 20 Layers
for i in [64, 128, 256, 512]:
    df = pd.read_csv("{}tested_mlp_15_20_{}_history.csv".format(root_csv_folder, i), sep=";")
    df = df[df.epochs == 400]
    results_dataframes_list.append(df)
    results_dataframes_desc.append("mlp_15_20_{}".format(i))

# MLP Without regularisation results 15 to 20 Layers
for i in [64, 128, 256, 512]:
    df = pd.read_csv("{}tested_mlp_20_30_{}_history.csv".format(root_csv_folder, i), sep=";")
    df = df[df.epochs == 400]
    results_dataframes_list.append(df)
    results_dataframes_desc.append("mlp_20_30_{}".format(i))

### Extraction des informations interessantes :
- Pire résultat sur le train
- Pire résultat sur la val
- Meilleur résultat sur le train
- Meilleur résultat sur la val 
- Moyenne des résultats sur le train 
- Moyenne des résultats sur la val 
- Mediane des résultat sur le train 
- Mediane des résultat sur la val

In [4]:
def print_statistics(history_df, history_df_desc):
    # Calculating statistics on all models
    if(not history_df.empty):
        sorted_train_accuracy = np.sort(history_df.train_accuracy)
        sorted_val_accuracy = np.sort(history_df.val_accuracy)

        max_val_accuracy = sorted_val_accuracy[-1]
        second_max_val_accuracy = sorted_val_accuracy[-2]
        min_val_accuracy = history_df.val_accuracy.min()
        mean_val_accuracy = history_df.val_accuracy.mean()
        median_val_accuracy = history_df.val_accuracy.median()

        max_train_accuracy = sorted_train_accuracy[-1]
        second_max_train_accuracy = sorted_train_accuracy[-2]
        min_train_accuracy = history_df.train_accuracy.min()
        mean_train_accuracy = history_df.train_accuracy.mean()
        median_train_accuracy = history_df.train_accuracy.median()

        # Printing models statistics
        print()
        print("########### Models {} ###########".format(history_df_desc))
        print(history_df.shape)
        print("Pire résultat sur le train : ", min_train_accuracy)
        print("Pire Résultat sur la val : ", min_val_accuracy)

        print("Meilleur résultat sur le train : ", max_train_accuracy)
        print("Meilleur résultat sur la val : ", max_val_accuracy)

        print("Moyenne des résultats sur le train : ", mean_train_accuracy)
        print("Moyenne des résultats sur la val : ", mean_val_accuracy)

        print("Mediane des résultat sur le train : ", median_train_accuracy)
        print("Mediane des résultat sur la val : ", median_val_accuracy)

def filter_on_median_val_accuracy(results_dataframes_list):
    filtered_results_dataframes = []
    
    for i in range(len(results_dataframes_list)):

        # Calculating statistics on all models
        history_df = results_dataframes_list[i]
        history_df_desc = results_dataframes_desc[i]

        median_val_accuracy = history_df.val_accuracy.median()
        mean_val_accuracy = history_df.val_accuracy.mean()
        # Filtering models to get the best ones (those whose val accuracy is higher then the median)
        
        # Si la médiane est inférieure à la moyenne on enlève cette archi, car la majorité des modèles on une mauvaise
        # accuracy, et la moyenne est gonflée par seulement une petite partie des modèles. Ce qui indique que que l'archi
        # choisie n'est pas interessante
#         if(mean_val_accuracy < median_val_accuracy):
        filtered_history = history_df[history_df.val_accuracy > median_val_accuracy]
#         else:
#         filtered_history = pd.DataFrame(columns=history_df.columns)
        filtered_results_dataframes.append(filtered_history)
        
    
    return filtered_results_dataframes


def filter_on_mean_train_accuracy(results_dataframes_list):
    filtered_results_dataframes = []
    
    for i in range(len(results_dataframes_list)):

        # Calculating statistics on all models
        history_df = results_dataframes_list[i]
        history_df_desc = results_dataframes_desc[i]

#         median_train_accuracy = history_df.train_accuracy.median()
        mean_train_accuracy = history_df.train_accuracy.mean()
        # Filtering models to get the best ones (those whose val accuracy is higher then the median)
        
        # Si la médiane est inférieure à la moyenne on enlève cette archi, car la majorité des modèles on une mauvaise
        # accuracy, et la moyenne est gonflée par seulement une petite partie des modèles. Ce qui indique que que l'archi
        # choisie n'est pas interessante
        filtered_history = history_df[history_df.train_accuracy > mean_train_accuracy]
        filtered_results_dataframes.append(filtered_history)
        
    
    return filtered_results_dataframes

def concat_list_of_dataframes(list_df):
    return pd.concat(list_df)

In [5]:
for i in range(len(results_dataframes_list)):
    history = results_dataframes_list[i]
    history_desc = results_dataframes_desc[i]
    print_statistics(history, history_desc)


########### Models mlp_5_10_64 ###########
(10, 22)
Pire résultat sur le train :  0.1
Pire Résultat sur la val :  0.1
Meilleur résultat sur le train :  0.52582
Meilleur résultat sur la val :  0.4642
Moyenne des résultats sur le train :  0.299574
Moyenne des résultats sur la val :  0.27585000000000004
Mediane des résultat sur le train :  0.28749
Mediane des résultat sur la val :  0.2706

########### Models mlp_5_10_128 ###########
(10, 22)
Pire résultat sur le train :  0.1
Pire Résultat sur la val :  0.1
Meilleur résultat sur le train :  0.6802199999999999
Meilleur résultat sur la val :  0.4826
Moyenne des résultats sur le train :  0.48029599999999995
Moyenne des résultats sur la val :  0.39284
Mediane des résultat sur le train :  0.54654
Mediane des résultat sur la val :  0.46055

########### Models mlp_5_10_256 ###########
(10, 22)
Pire résultat sur le train :  0.1
Pire Résultat sur la val :  0.1
Meilleur résultat sur le train :  0.8409200000000001
Meilleur résultat sur la val :  0.4

###### On ne garde que les modèles dont les résultats sur la validations sont supérieurs à la médiane des résultats de tous les modèles (d'une même architecture) testés 

In [6]:
filtered_results_dataframes = filter_on_median_val_accuracy(results_dataframes_list)

In [7]:
for i in range(len(filtered_results_dataframes)):
    filtered_history = filtered_results_dataframes[i]
    filtered_history_desc = results_dataframes_desc[i]
    print_statistics(filtered_history, filtered_history_desc)


########### Models mlp_5_10_64 ###########
(5, 22)
Pire résultat sur le train :  0.47498
Pire Résultat sur la val :  0.4412
Meilleur résultat sur le train :  0.52582
Meilleur résultat sur la val :  0.4642
Moyenne des résultats sur le train :  0.4991479999999999
Moyenne des résultats sur la val :  0.45170000000000005
Mediane des résultat sur le train :  0.49518
Mediane des résultat sur la val :  0.4499

########### Models mlp_5_10_128 ###########
(5, 22)
Pire résultat sur le train :  0.5391600000000001
Pire Résultat sur la val :  0.4661
Meilleur résultat sur le train :  0.6382
Meilleur résultat sur la val :  0.4826
Moyenne des résultats sur le train :  0.5760759999999999
Moyenne des résultats sur la val :  0.47286
Mediane des résultat sur le train :  0.5705399999999999
Mediane des résultat sur la val :  0.4698

########### Models mlp_5_10_256 ###########
(5, 22)
Pire résultat sur le train :  0.5229600000000001
Pire Résultat sur la val :  0.4604
Meilleur résultat sur le train :  0.77586

###### On fusionne tous les dataframes en une seule (étant donné qu'il y en a une par structure de modèle)

In [8]:
merged_df = concat_list_of_dataframes(filtered_results_dataframes)

In [9]:
print_statistics(merged_df, "mlp_without_regularisation")


########### Models mlp_without_regularisation ###########
(79, 22)
Pire résultat sur le train :  0.24884
Pire Résultat sur la val :  0.2473
Meilleur résultat sur le train :  1.0
Meilleur résultat sur la val :  0.4902
Moyenne des résultats sur le train :  0.6918189873417723
Moyenne des résultats sur la val :  0.4486696202531646
Mediane des résultat sur le train :  0.6338199999999999
Mediane des résultat sur la val :  0.4556


### Première analyse des résultats obtenus :

#### Nous remarquons donc sur les 78 meilleurs modèles, nous tournons autours des 45% de résultat sur la validation, et 65% sur le train. De plus, nous avons atteint une limite de taille à lancer, étant donné que nos fits crashent pour manque de mémoire GPU quand on essaye de s'attaquer à des modèles d'environs 30 couches cachées. 

#### Ces résultats nous montrent que les meilleurs modèles overfittent. Nous allons donc essayer d'appliquer des techniques de régularisation dessus, en prenant les meilleurs sur ces 33 modèles selectionnés. Mais avant cela, nous allons essayer de réduire le nombre de modèles à tester on ne prenant que les plus intéressants.

## Selection des meilleurs modèles sur la val sur les 64 selectionnés:
    Pour cela, nous n'allons prendre que les modèles dont les résultats sur l'entraînement sont supérieurs à la médiane des résultats de tous les modèles compris (toutes arachitectures confondues).

In [10]:
best_val_models = filter_on_median_val_accuracy([merged_df])[0]
best_val_models = best_val_models.sort_values('activation_couches', ascending=False)
print_statistics(best_val_models, "Best MLP models")


########### Models Best MLP models ###########
(39, 22)
Pire résultat sur le train :  0.49518
Pire Résultat sur la val :  0.4557
Meilleur résultat sur le train :  1.0
Meilleur résultat sur la val :  0.4902
Moyenne des résultats sur le train :  0.67856
Moyenne des résultats sur la val :  0.466625641025641
Mediane des résultat sur le train :  0.6226
Mediane des résultat sur la val :  0.4661


###### On voit donc que nous avons 38 modèles qui pourrait être intéressants sur la validation, mais nous allons maintenant en selectionné, entre ceux là, les meilleurs sur le train. 

## Selection des meilleurs modèles sur le train sur les 32 restants:

In [11]:
best_models = filter_on_mean_train_accuracy([best_val_models])[0]
best_models = best_models.sort_values('val_accuracy', ascending=False)
print_statistics(best_models, "Best MLP models")


########### Models Best MLP models ###########
(15, 22)
Pire résultat sur le train :  0.68012
Pire Résultat sur la val :  0.4557
Meilleur résultat sur le train :  1.0
Meilleur résultat sur la val :  0.4735
Moyenne des résultats sur le train :  0.842788
Moyenne des résultats sur la val :  0.46584
Mediane des résultat sur le train :  0.8357600000000001
Mediane des résultat sur la val :  0.4669


###### Nous avons donc 14 modèles interessants qui semblent intéressants

In [12]:
best_models

,nb_couches,profondeurs_couches,activation_couches,activation_output,dropout,nb_dropout,indexes_dropout,valeur_dropout,l1l2_couches,l1l2_output,...,nb_l1l2,indexes_l1l2,loss,optimizer,metrics,epochs,model_id,train_accuracy,val_accuracy,dossier
6,11,512 512 512 512 512 512 512 512 512 512 512,softplus,softmax,False,0,NaN,0.0,False,False,...,0,NaN,sparse_categorical_crossentropy,Adam,sparse_categorical_accuracy,400,5762652,0.71452,0.4735,20200123
7,16,512 512 512 512 512 512 512 512 512 512 512 51...,softplus,softmax,False,0,NaN,0.0,False,False,...,0,NaN,sparse_categorical_crossentropy,Adam,sparse_categorical_accuracy,400,4143798,1.00000,0.4730,20200123
0,14,256 256 256 256 256 256 256 256 256 256 256 25...,selu,softmax,False,0,NaN,0.0,False,False,...,0,NaN,sparse_categorical_crossentropy,Adam,sparse_categorical_accuracy,400,7574331,0.69438,0.4704,20200122
2,14,512 512 512 512 512 512 512 512 512 512 512 51...,relu,softmax,False,0,NaN,0.0,False,False,...,0,NaN,sparse_categorical_crossentropy,Adam,sparse_categorical_accuracy,400,2810928,0.98868,0.4694,20200122
5,9,256 256 256 256 256 256 256 256 256,selu,softmax,False,0,NaN,0.0,False,False,...,0,NaN,sparse_categorical_crossentropy,Adam,sparse_categorical_accuracy,400,9815569,0.70862,0.4683,20200122
7,15,128 128 128 128 128 128 128 128 128 128 128 12...,selu,softmax,False,0,NaN,0.0,False,False,...,0,NaN,sparse_categorical_crossentropy,Adam,sparse_categorical_accuracy,400,568701,0.68012,0.4682,20200122
8,9,512 512 512 512 512 512 512 512 512,relu,softmax,False,0,NaN,0.0,False,False,...,0,NaN,sparse_categorical_crossentropy,Adam,sparse_categorical_accuracy,400,1855666,0.68112,0.4676,20200122
4,18,512 512 512 512 512 512 512 512 512 512 512 51...,relu,softmax,False,0,NaN,0.0,False,False,...,0,NaN,sparse_categorical_crossentropy,Adam,sparse_categorical_accuracy,400,2811332,0.99686,0.4669,20200123
4,8,256 256 256 256 256 256 256 256,relu,softmax,False,0,NaN,0.0,False,False,...,0,NaN,sparse_categorical_crossentropy,Adam,sparse_categorical_accuracy,400,6310408,0.77586,0.4668,20200122
9,7,512 512 512 512 512 512 512,relu,softmax,False,0,NaN,0.0,False,False,...,0,NaN,sparse_categorical_crossentropy,Adam,sparse_categorical_accuracy,400,4390588,0.98032,0.4657,20200122


In [ ]:
best_models.to_csv("{}best_mlp_without_regularisation.csv".format(root_csv_folder), sep=';',header=True, index=False)

## Conclusions premiers tests:

##### Avec cette première vague de tests, nous avons vu que des MLP, mêmes assez gros (du moins, les plus gros qu'on a pu lancer (Maximum 25 à 30 couches à 512 neurones par couches)) ne donnent pas de résultats intéressant sur la validation (un maximum de 48%).

##### Cependant, on a aussi remarqué que tous ces modèles on tendance à overfitter. Du coup on va essayer d'appliquer des techniques de régulatisation pour voir si on peu tirer de meilleurs résultats avec les meilleurs modèles qu'on a réussi à avoir.

##### Nous avons selectionnés deux modèles pour poursuivre les tests : le meilleur modèle qu'on a obtenu (11 couches à 512 neuronnes par couche avec une softplus pour l'activation des couches cachées et une softmax pour l'output) et un modèle qui overfitte beaucoup  (5 couches à 512 neurones par couche, avec la relu pour les couches cachées et la softmax pour l'output). 

# --------------------------------------------------------------------------------------------------------------

# Tests MLP avec techniques de régularisations:

### Modèle à 5 couches:
###### Résultats sans régularisation : 0.46 en validation et 0.87 en train

##### Avec Régularisation L2:

- L2 à 0.0005 : 0.45 sur la validation et 0.9 sur le train => même résultat que sans régularisation
- L2 à 0.01: 0.5 sur la validation et 0.75 sur le train (qui était à 0.87 sans régularisation) mais on a toujour un overfitting => Un peu mieux, mais on peu essaye de régulariser encore plus
- L2 à 0.2: 0.31 sur la validation et le train => on a trop pénalisé les poids

##### Avec Régularisation L1 + L2:

- L2 à 0.01 et L1 à 0.01: 0.31 sur la validation et le train => on a trop pénalisé les poids

### Modèle à 11 couches:

###### Résultats sans régularisation : 0.4735 en validation et 0.71 en train

##### Avec Régularisation L2 sur les couches cachées:

- L2 à 0.01: 0.37 en validation et 0.38 en train => Trop pénalisé
- L2 à 0.001: 0.58 sur le train, et sur la validation monte jusqu'à 0.53 sur certaine epochs. par contre, tout le long du train, l'accuracy s'effondre à plusieurs reprises pour ensuite remonter (comme le montre le logs du fit). On retrouvera ce comportement sur plusieurs autres entrau=inement du même modèle avec d'autres hyperparamètres(comme avecla l2 à 0.0005). On n'arrive pas à expliquer cela.
- L2 à 0.0005: Monte jusqu'à 0.54 en validation sur certaines epochs et à 0.95 sur le train. => Overfit, nous avons donc essayé d'ajouter une L1 à ce même modèle.
- L2 à 0.002: Validation à 0.53 et 0.6 sur le train, Accuracy retombe de temps en temps

In [15]:
%load_ext tensorboard
%tensorboard --logdir ".\\logs\\mlp_best_models_with_l2\\20200124\\mlp_best_models_with_l2_2848849.log"

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 4396.

##### Avec Régularisation L2 et L1 sur les couches cachées:

- l2 à 0.0005 et L1 à 0.0001: 0.5 en validation et 0.54 ssur le train

##### Avec Régularisation L2 sur les couches cachées et output:

- L2 à 0.001: 0.5 sur la validation et 0.55 sur le train
- L2 à 0.0005: 0.5 sur la validation et 0.55 sur le train

##### Avec Régularisation L2 sur les couches cachées et Dropout:

- L2 à 0.001 et dropout à 0.1: 0.43 sur la validation et 0.47 sur le train
- L2 à 0.001 et dropout à 0.2: 0.29 sur la validation et le train
- L2 à 0.001 et dropout à 0.3: 0.27 sur la validation et le train

##### Avec Régularisation Dropout sur les inputs:

- Dropout à 0.2: 0.89 sur le train et 0.45 sur la val

# -------------------------------------------------------------------------------------------------------------

# Conclusion MLP:

### Résumé de résultats obtenus:

##### Même en utilisations différents hyperparamètres et techniques de régularisation, avec de simples MLP, nous avons vu que l'accuracy maximale que nous avons pu atteindre sur la validation se situe autours des 54%. Etant donné que nous avons atteint nos limites en matière de ressources matérielles, nous allons essayer d'utiliser d'autres types de modèles sur le dataset pour voir ce que cela va donner. 